## Decision variables:
1. q[i, j, k] the quantities of product i (i=0, LCD42; i=1, LCD32) being shipped from 
ODM j (j=0, ODM1, ...,j=6, ODM7) via mode k (k=0, Regular Air; k=1, Air Express; k=2, Road; k=3, Road LTL; k=4, Road Network; k=5, Rail;
k=6, Water).
2. y[j] binary, whether ODM j is producing, =1 produce, =0 not produce
3. z[i, j, k] binary, whether mode k from ODM j for product i is used to ship goods, =1 used, =0 not used.

## Parameters:
1. weight[i]: weights of LCD42 and LCD32
2. distance[i, j]: distances between ODM j and DC
3. mode_category[c]: a list of categories including air, road, rail, water
4. f_shipping_cost[i, j, k]: shipping costs of product i being shipped from ODM j via mode k
5. f_min_units_mode[i, j, k]: the minimum units to be shipped of product i via mode category c
6. f_production_cost[i, j]: production cost for product i in ODM j
7. f_emission[k]: emission(kg) released per km per ton for mode k
8. total_demand[i]: demand for product i
9. min_production: at least 200000 units to be produced at any ODM, for LCD42 and LCD32 combined
10. max_production: at most 600000 units to be produced at any ODM except ODM 1 and 2, for LCD42 and LCD32 combined
11. model.I, model.J, model.K, model.C: ranges of number of products, ODMs, modes, and categories
12. valid_combinations: valid (i, j, k) in f_shipping_cost array

## Objective funtion for Q1
$$ Minimise: Obj = \sum_{i=0}^{1}\ \sum_{j=0}^{6}\ ProductionCost(i,j) \times\ \sum_{k=0}^{6} q(i,j,k)\ + \sum_{i=0}^{1}\ \sum_{j=0}^{6}\ \sum_{k=0}^{6} q(i,j,k) \times\ ShippingCost(i,j,k) \times\ weight(i)\ $$

## Constraints for Q1:
1. Shipping constraints:
$ for \quad i,j,k \in ValidCombination: \quad q(i,j,k) \geq MinimumUnitsMode(i,j,k) \times z(i,j,k) \quad and \quad q(i,j,k) \leq MinimumUnitsMode(i,j,k) \times z(i,j,k)$
2. Enforce usage constraint: $ for \quad i \in I, c \in C: \quad \sum_{j=0}^{6} \sum_{k=0}^{krange} z(i,j,k) \geq 1$
3. Production constraints: $ for \quad j \in J: \quad \sum_{i=0}^{1} \sum_{k=0}^{6} q(i, j, k) \geq MinProduction \times y(j) \quad and \quad \sum_{i=0}^{1} \sum_{k=0}^{6} q(i, j, k) \leq MaxProduction \times y(j)$
4. Demand constraints: $ for \quad i \in I: \quad \sum_{j=0}^{6}\ \sum_{k=0}^{6} q(i, j, k) \geq TotalDemand(i) $

In [126]:
import numpy as np
from pyomo.environ import *
import pandas as pd

In [127]:
# read file
file_name='DHL_final.xlsx'
# shipping cost
shipping_cost = pd.read_excel(file_name, "ShippingCost", index_col=0) 
# minimum units to be shipped by category
min_trans_unit_category = pd.read_excel(file_name, "MiniNumberofShipping_Category", index_col = 0)
# minimum units to be shipped by mode
min_trans_unit_mode = pd.read_excel(file_name, "MiniNumberofShipping_Mode", index_col = 0)
# distance between ODMs and DC
distance = pd.read_excel(file_name, "Distance", index_col = 0)
# production cost
production_cost = pd.read_excel(file_name, "ProductionCost", index_col = 0)
# emission(kg) per tom per km
emission_kg_per_ton_km = pd.read_excel(file_name, "EmissionCost", index_col = None)
# mode categories. air has regular air and air express; road has road, road LTL, and Road Network; rail has only rail and so is water
mode_category = ["air", "road", "rail", "water"]
# total units to be shipped 
total_demand = [920000, 530000]
# weights of LCD42 and LCD32
weight = [0.022, 0.0165]
# minimum and maximum production of LCD42 and LCD32 combined at any ODM
min_production = 200000
max_production = 600000

In [128]:
# save the names of products, ODMs, and modes
product_name = ['LCD42', 'LCD32']
ODM_name = ['1', '2', '3', '4', '5', '6', '7']
mode_name = list(shipping_cost.columns)
mode_category_name = list(min_trans_unit_category)

# save the length of them
product_num = len(product_name)
ODM_num = len(ODM_name)
mode_num = len(mode_name)
mode_category_num = len(mode_category)

In [129]:
# Here we are going make two 3-d array, shipping cost and minimum units to be shipped via mode.
# Turn a dataframe into an array
shipping_cost_np = shipping_cost.to_numpy()
# take shipping costs out
L42_shipping_cost = shipping_cost_np[:-2, :]
L32_shipping_cost = shipping_cost_np[-2:, :]
# build 3d array filled with nan, and then put our shipping cost data in it
f_shipping_cost = np.full((2, ODM_num, mode_num), np.nan)
# put shipping cost into this nan array. It is now a 3d shipping cost array.
f_shipping_cost[0, :L42_shipping_cost.shape[0], :L42_shipping_cost.shape[1]] = L42_shipping_cost
f_shipping_cost[1, :L32_shipping_cost.shape[0], :L32_shipping_cost.shape[1]] = L32_shipping_cost

# Similarily, we make a 3d array of minimum units to be shipped by each mode
min_trans_unit_mode_np = min_trans_unit_mode.to_numpy()
f_min_units_mode = np.full((2, ODM_num, mode_num), np.nan)
min_L42_unit = min_trans_unit_mode_np[0, :]
min_L32_unit = min_trans_unit_mode_np[1, :]
for i in range(ODM_num):
    f_min_units_mode[0, i, :] = min_L42_unit
for i in range(2):
    f_min_units_mode[1, i, :] = min_L32_unit


# Make final production, distance, and emission tables
f_production_cost = production_cost.to_numpy()
f_distance = distance.to_numpy()
f_emission = emission_kg_per_ton_km.to_numpy()[0, :]

In [130]:
# Start to build our model
model = ConcreteModel()

In [131]:
# units to be shipped for product i, ODM j, mode k
model.q = Var(range(product_num), range(ODM_num), range(mode_num), domain = NonNegativeIntegers, initialize = 0)
# Introduce a binary variable y. It can be used to formulate the production constraint
model.y = Var(range(ODM_num), domain = Binary)
# Introduce a binary variable z. It can be used to formulate the shipping constraint
model.z = Var(range(product_num), range(ODM_num), range(mode_num), domain = Binary)

In [132]:
# We have nan values in the f_shipping_cost, so we need to find valid combinations for i, j, k. 
# This will help us simplify following steps.
valid_combinations = []
for i in range(product_num):
    for j in range(ODM_num):
        for k in range(mode_num):
            if not np.isnan(f_shipping_cost[i, j, k]):
                valid_combinations.append((i, j, k))

# Create sets for products, ODMs, modes, and valid combinations
model.I = Set(initialize=range(product_num))
model.J = Set(initialize=range(ODM_num))
model.K = Set(initialize=range(mode_num))
model.C = Set(initialize=range(mode_category_num))
model.ValidCombinations = Set(within=model.I * model.J * model.K, initialize=valid_combinations)

In [133]:
# Define objective function. 
# We need to use if not condition to exclude all the nan values in production_costs because we can't use the valid combinations
def obj(model):
    # Calculate production costs for valid combinations
    production_costs = sum(f_production_cost[i, j] * (sum(model.q[i, j, k] if not np.isnan(f_shipping_cost[i, j, k]) else 0
                                                          for k in model.K)) 
                           if not np.isnan(f_production_cost[i, j]) else 0
                           for i in model.I for j in model.J)
    
    # Calculate shipping costs for valid combinations
    shipping_costs = sum(f_shipping_cost[i, j, k] * weight[i] * model.q[i, j, k]
                         for i, j, k in valid_combinations)
    
    return production_costs + shipping_costs

model.Obj = Objective(rule=obj, sense=minimize)

In [134]:
# Shipping constraint
def min_shipping_cons(model, i, j, k):
    return model.q[i, j, k] >= f_min_units_mode[i, j, k] * model.z[i, j, k]

def max_shipping_cons(model, i, j, k):
    bigM = 1000000  # Large number, effectively serving as an upper bound
    return model.q[i, j, k] <= bigM * model.z[i, j, k]

# Apply constraints over ValidCombinations to ensure they're only defined for non-NaN cost indices
model.Min_ShippingCons = Constraint(model.ValidCombinations, rule=min_shipping_cons)
model.Max_ShippingCons = Constraint(model.ValidCombinations, rule=max_shipping_cons)

In [135]:
# Create a function to determine the range of k for different categories.
def determine_k_range(c):
    # Determines the range of mode indices for a given transport category.
    if c == 0:  # air
        return range(2)  # modes 0 and 1 are air transport
    elif c == 1:  # road
        return range(2, 5)  # modes 2, 3, and 4 are road transport
    elif c == 2:  # rail
        return [5]  # mode 5 is rail transport
    elif c == 3:  # water
        return [6]  # mode 6 is water transport
    else:
        raise ValueError("Invalid category index")

In [136]:
# We assume that to ensure enough resilience, we need to use every category for each product at least once
# This constrain ensures our assumptions.
def enforce_product_category_usage(model, i, c):
    # This constraint ensures the binary variable for category usage is set to 1 if any mode within the category is used
    return sum(model.z[i, j, k] if not np.isnan(f_shipping_cost[i, j, k]) else 0 
               for j in model.J for k in determine_k_range(c)) >= 1 

# Adding the constraints to the model
model.EnforceCategoryUsage = Constraint(model.I, model.C, rule=enforce_product_category_usage)

In [137]:
# Production constraint
# We need to have a minimum order of 200,000 per ODM, for LCD42 and LCD32 combined.
# Therefore, we will use j as index instead of (i, j, k), we cannot use validcombinations
# We need to filter nan values in the f_shipping_cost.
def min_production_cons(model, j):
    return (sum(model.q[i, j, k] if not np.isnan(f_shipping_cost[i, j, k]) else 0 
                for i in model.I for k in model.K)) >= min_production * model.y[j]
def max_production_cons(model, i, j):
    return (sum(model.q[i, j, k] 
                       if not np.isnan(f_shipping_cost[i, j, k]) else 0 for k in model.K)) <= max_production * model.y[j]
model.Min_ProductionCons = Constraint(model.J, rule=min_production_cons)
model.Max_ProductionCons = Constraint(model.I, model.J, rule=max_production_cons)

In [138]:
# Demand constraint
def demand_cons(model, i):
    return sum(model.q[i, j, k]
               if not np.isnan(f_shipping_cost[i, j, k]) else 0
               for j in model.J for k in model.K) >= total_demand[i]

model.DemandCons = Constraint(model.I, rule=demand_cons)

In [139]:
# solve this problem using glpk and save the results
solver = SolverFactory('glpk')
results = solver.solve(model,load_solutions=False,tee=False)

In [140]:
# check the status is right
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    model.solutions.load_from(results)
else:
    print("Solve failed.")

In [141]:
# print the results
print("total minimum cost:", model.Obj())
for i in range(product_num):
    for j in range(ODM_num):
        for k in range(mode_num):
            if value(model.q[i, j, k]) > 0:
                print("The quantities to be shipped for product", product_name[i], "ODM", ODM_name[j], "via mode", mode_name[k], 
                      value(model.q[i, j, k]))
            else:
                continue

total minimum cost: 2999985597.1
The quantities to be shipped for product LCD42 ODM 1 via mode Regular Air 46000.0
The quantities to be shipped for product LCD42 ODM 1 via mode Road Network 92000.0
The quantities to be shipped for product LCD42 ODM 1 via mode Rail 138000.0
The quantities to be shipped for product LCD42 ODM 1 via mode Water 44000.0
The quantities to be shipped for product LCD42 ODM 4 via mode Water 600000.0
The quantities to be shipped for product LCD32 ODM 1 via mode Regular Air 53000.0
The quantities to be shipped for product LCD32 ODM 1 via mode Road Network 79500.0
The quantities to be shipped for product LCD32 ODM 1 via mode Rail 79500.0
The quantities to be shipped for product LCD32 ODM 1 via mode Water 318000.0


In [121]:
emission = 0
for i in range(product_num):
    for j in range(ODM_num):
        for k in range(mode_num):
            if value(model.q[i, j, k]) > 0:
                emission += f_emission[k] * value(model.q[i, j, k]) * weight[i] * f_distance[i,j]
                
            else:
                continue
print("The CO2 emission(kg) is:", emission)

The CO2 emission(kg) is: 7944511.033799999
